In [1]:
import xarray as xr
import rioxarray
import numpy as np
from rasterio.enums import Resampling
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import zarr
import fsspec
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"
data_dir = "/data/ABOVE/ABoVE_Final_Data/air_temp/Tundra_Greeness_Temp_Trends_1893/data/"
out_dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/swi/"

/tmp/ipykernel_597609/1263652735.py:6: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


Resample ERA5 air temperature


In [2]:
# target_image = xr.open_dataset(
#     dir +
#     "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_max.nc"
# )["LAI"].isel(time=5)
# target_image = target_image.rio.write_crs("EPSG:4326").rename({
#     "latitude": "y",
#     "longitude": "x"
# })
# geodf = gpd.read_file("../data/shp_files/CoreDomain_geographic.shp")
# source_image = xr.open_dataset("../data/t2m_era5.nc")["t2m"].isel(expver=0)
# source_image = source_image.rio.write_crs(target_image.rio.crs)
# resampled = source_image.rio.reproject_match(target_image,
#                                              resampling=Resampling.average)
# clipped = resampled.rio.clip(geodf.geometry).rename({"y": "lat", "x": "lon"})
# t2m_annual_resample_max = clipped.groupby("time.year").max().rename(
#     {"year": "time"})
# t2m_annual_resample_max['time'] = pd.date_range("1984", "2024", freq="A")
# t2m_annual_resample_max = t2m_annual_resample_max.sel(
#     time=slice("1984", "2022"))

# t2m_annual_resample_mean = clipped.groupby("time.year").mean().rename(
#     {"year": "time"})
# t2m_annual_resample_mean['time'] = pd.date_range("1984", "2024", freq="A")
# t2m_annual_resample_mean = t2m_annual_resample_mean.sel(
#     time=slice("1984", "2022"))

# t2m_summer = clipped.where(clipped.time.dt.month.isin([6, 7, 8]), drop=True)
# t2m_summer_resample_mean = t2m_summer.groupby("time.year").mean().rename(
#     {"year": "time"})
# t2m_summer_resample_mean['time'] = pd.date_range("1984", "2024", freq="A")
# t2m_summer_resample_mean = t2m_annual_resample_mean.sel(
#     time=slice("1984", "2022"))

# t2m_summer_resample_max = t2m_summer.groupby("time.year").max().rename(
#     {"year": "time"})
# t2m_summer_resample_max['time'] = pd.date_range("1984", "2024", freq="A")
# t2m_summer_resample_max = t2m_annual_resample_max.sel(
#     time=slice("1984", "2022"))

# t2m_annual_resample_max.to_netcdf("../data/t2m_annual_resample_max.nc")
# t2m_annual_resample_mean.to_netcdf("../data/t2m_annual_resample_mean.nc")
# t2m_summer_resample_max.to_netcdf("../data/t2m_summer_resample_max.nc")
# t2m_summer_resample_mean.to_netcdf("../data/t2m_summer_resample_mean.nc")

In [2]:
ds = xr.open_dataset("../data/ERA5.nc")

In [100]:
target_image = xr.open_dataset(
    "../data/lai_annual_resample_max.nc")["LAI"].isel(time=5)
target_image = target_image.rio.write_crs("EPSG:4326").rename({
    "latitude": "y",
    "longitude": "x"
})
geodf = gpd.read_file("../data/shp_files/CoreDomain_geographic.shp")

# vars = ["d2m", "t2m", "ssr", "tp", "swvl1"]
vars=["tp", "swvl1"]
for var in vars:
    print(var)

    source_image = ds[var].isel(expver=0)
    source_image = source_image.rio.write_crs(target_image.rio.crs)
    resampled = source_image.rio.reproject_match(target_image,
                                                 resampling=Resampling.average)
    clipped = resampled.rio.clip(geodf.geometry).rename({
        "y": "lat",
        "x": "lon"
    })
    if (var=="tp")|(var=="swvl1"):
        var_total_annual = clipped.groupby("time.year").sum("time").rename(
        {"year": "time"}) 
        var_total_annual['time'] = pd.date_range("1983", "2024", freq="A")
        var_total_annual = var_total_annual.sel(
        time=slice("1984", "2021"))
        var_total_annual.to_netcdf("../data/"+var+"_annual_resample_total.nc")
        continue

    var_annual_resample_max = clipped.groupby("time.year").max().rename(
        {"year": "time"})
    var_annual_resample_max['time'] = pd.date_range("1983", "2024", freq="A")
    var_annual_resample_max = var_annual_resample_max.sel(
        time=slice("1984", "2021"))

    var_annual_resample_mean = clipped.groupby("time.year").mean().rename(
        {"year": "time"})
    var_annual_resample_mean['time'] = pd.date_range("1983", "2024", freq="A")
    var_annual_resample_mean = var_annual_resample_mean.sel(
        time=slice("1984", "2021"))
    var_annual_resample_max.to_netcdf("../data/" + var +
                                      "_annual_resample_max.nc")
    var_annual_resample_mean.to_netcdf("../data/" + var +
                                       "_annual_resample_mean.nc")


tp
swvl1


In [15]:
T = ds["t2m"].isel(expver=0)-273.15
Td = ds["d2m"].isel(expver=0)-273.15

c1 = 0.611
c2=17.5
c3 = 240.978

es = c1*np.exp((c2*T)/(c3+T))
ea = c1*np.exp((c2*Td)/(c3+Td))
VPD = es - ea 

In [26]:
target_image = xr.open_dataset(
    "../data/lai_annual_resample_max.nc")["LAI"].isel(time=5)
target_image = target_image.rio.write_crs("EPSG:4326").rename({
    "latitude": "y",
    "longitude": "x"
})
geodf = gpd.read_file("../data/shp_files/CoreDomain_geographic.shp")

source_image = VPD
source_image = source_image.rio.write_crs(target_image.rio.crs)
resampled = source_image.rio.reproject_match(target_image,
                                                resampling=Resampling.average)
clipped = resampled.rio.clip(geodf.geometry).rename({
    "y": "lat",
    "x": "lon"
})
clipped = clipped.where(clipped<10)

vpd_annual_resample_max = clipped.groupby("time.year").max().rename(
    {"year": "time"})
vpd_annual_resample_max['time'] = pd.date_range("1983", "2024", freq="A")
vpd_annual_resample_max = vpd_annual_resample_max.sel(
    time=slice("1984", "2021"))

vpd_annual_resample_mean = clipped.groupby("time.year").mean().rename(
    {"year": "time"})
vpd_annual_resample_mean['time'] = pd.date_range("1983", "2024", freq="A")
vpd_annual_resample_mean = vpd_annual_resample_mean.sel(
    time=slice("1984", "2021"))
vpd_annual_resample_max.to_netcdf("../data/vpd_annual_resample_max.nc")
vpd_annual_resample_mean.to_netcdf("../data/vpd_annual_resample_mean.nc")

/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/rioxarray/raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


In [2]:
ds = xr.open_dataset("../../data/raw_data/co2/CO2_1deg_month_1850-2013.nc")
ds["value"].sel(Times=slice("1984","2013"))["Times"]
ds = xr.open_dataset("../../data/raw_data/co2/CO2_1deg_month_1850-2013.nc")
co2= ds["value"].sel(Times=slice("1984","2013")).rename({"LatDim":"y","LonDim":"x","Times":"time"})
target_image = xr.open_dataset(
    "../data/lai_annual_resample_max.nc")["LAI"].isel(time=5)
target_image = target_image.rio.write_crs("EPSG:4326").rename({
    "latitude": "y",
    "longitude": "x"
})
geodf = gpd.read_file("../data/shp_files/CoreDomain_geographic.shp")

source_image = co2
source_image = source_image.rio.write_crs(target_image.rio.crs)
resampled = source_image.rio.reproject_match(target_image,
                                                resampling=Resampling.average)
clipped = resampled.rio.clip(geodf.geometry).rename({
    "y": "lat",
    "x": "lon"
})
clipped = clipped.where(clipped<600)
co2_annual_resample_mean = clipped.groupby("time.year").mean().rename(
    {"year": "time"})

co2_annual_resample_mean['time'] = pd.date_range("1984", "2014", freq="A")
co2_annual_resample_mean.to_netcdf("../data/co2_annual_resample_mean.nc")

Process CMIP6 data

In [ ]:
target_image = xr.open_dataset(
    "../data/lai_annual_resample_max.nc")["LAI"].isel(time=5)
target_image = target_image.rio.write_crs("EPSG:4326").rename({
    "latitude": "y",
    "longitude": "x"
})
geodf = gpd.read_file("../data/shp_files/CoreDomain_geographic.shp")

In [2]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df_lai = df.query("activity_id=='CMIP' & variable_id == 'lai' & experiment_id == 'historical' ")

gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Lmon/lai/gn/v20190308/


In [ ]:
df_cesm2 =df_lai.query(" source_id=='CESM2' ") 
zstore = df_cesm2.zstore.values[0]
print(zstore)

# create a mutable-mapping-style interface to the store
mapper = fsspec.get_mapper(zstore)

# open it using xarray and zarr
ds = xr.open_zarr(mapper, consolidated=True)
lai = ds["lai"].rename({"lat":"y","lon":"x"}).sel(time=slice("1984","2013"))

source_image = lai
source_image = source_image.rio.write_crs(target_image.rio.crs)
resampled = source_image.rio.reproject_match(target_image,
                                                resampling=Resampling.average)
clipped = resampled.rio.clip(geodf.geometry).rename({
    "y": "lat",
    "x": "lon"
})
cesm2_lai_annual = clipped.groupby("time.year").mean().rename(
    {"year": "time"})
cesm2_lai_annual.to_netcdf("../data/cesm2_lai_annual.nc")

In [11]:
lai4g = xr.open_dataset("../data/lai4g.nc")["lai"]
source_image = lai4g
source_image = source_image.rio.write_crs(target_image.rio.crs)
resampled = source_image.rio.reproject_match(target_image,
                                                resampling=Resampling.average)
lai4g_annual_max = resampled.groupby("time.year").max().rename(
    {"year": "time","y": "lat", "x": "lon"})
lai4g_annual_max.to_netcdf("../data/lai4g_annual_max.nc")

In [ ]:
# target_image = xr.open_dataset(
#     dir +
#     "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_max.nc"
# )["LAI"].isel(time=5)
# target_image = target_image.rio.write_crs("EPSG:4326").rename({
#     "latitude": "y",
#     "longitude": "x"
# })

# # # Resampling autmn onset
# for i in np.arange(1985, 2014):
#     print(i)
#     source_image = xr.open_dataarray(data_dir + "arctic_swi_" + str(i) +
#                                      "_p500.tif",
#                                      engine="rasterio")
#     resampled = source_image.rio.reproject_match(target_image,
#                                                  resampling=Resampling.average)

#     resampled.squeeze().to_netcdf(out_dir + "resampled_" + str(i) + ".nc")

# fnames = []
# for i in np.arange(1985, 2014):
#     fnames.append(out_dir + "resampled_" + str(i) + ".nc")
# ds = xr.concat([xr.open_dataset(f)["band_data"] for f in fnames], dim="time")
# ds['time'] = pd.date_range("1985", "2014", freq="A")
# ds = ds.rename({"x": "lon", "y": "lat"}).drop("band")
# ds.to_netcdf("../data/swi.nc")